In [34]:
# All needed import
import numpy as np
import pandas as pd
from decouple import config

import mlflow

from typing import Tuple

In [35]:
# All import needed for the pre-processing
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [36]:
# All model import
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import AdaBoostRegressor, HistGradientBoostingRegressor

In [37]:
# import for the logs
from utils.mlflow_logs import log_confusion_matrix, log_fn_and_fp, log_f1_score, log_parameters

In [38]:
# Global variable
RANDOM_STATE = 42
SEED = 42
URI = config("URI")
EXPERIMENT_ID = "791343131907353119"

In [39]:
# Path to the datasets
DATA_PATH = "datasets/ds_salaries.csv"

In [40]:
MODEL_FUNCTION = {
    "lightGBM with poisson loss 1" : HistGradientBoostingRegressor(loss="poisson", learning_rate=0.01, tol=1e-7),
    "lightGBM with poisson loss 2" : HistGradientBoostingRegressor(loss="poisson", learning_rate=0.05, tol=1e-7),
    "lightGBM with poisson loss 3" : HistGradientBoostingRegressor(loss="poisson", learning_rate=0.1, tol=1e-7),
    "lightGBM with poisson loss 4" : HistGradientBoostingRegressor(loss="poisson", learning_rate=0.01, tol=1e-5),
    "lightGBM with poisson loss 5" : HistGradientBoostingRegressor(loss="poisson", learning_rate=0.05, tol=1e-5),
    "lightGBM with poisson loss 6" : HistGradientBoostingRegressor(loss="poisson", learning_rate=0.1, tol=1e-5),
    "lightGBM with abs loss 1" : HistGradientBoostingRegressor(loss="absolute_error", learning_rate=0.01, tol=1e-7),
    "lightGBM with abs loss 2" : HistGradientBoostingRegressor(loss="absolute_error", learning_rate=0.05, tol=1e-7),
    "lightGBM with abs loss 3" : HistGradientBoostingRegressor(loss="absolute_error", learning_rate=0.1, tol=1e-7),
    "lightGBM with abs loss 4" : HistGradientBoostingRegressor(loss="absolute_error", learning_rate=0.01, tol=1e-5),
    "lightGBM with abs loss 5" : HistGradientBoostingRegressor(loss="absolute_error", learning_rate=0.05, tol=1e-5),
    "lightGBM with abs loss 6" : HistGradientBoostingRegressor(loss="absolute_error", learning_rate=0.1, tol=1e-5),
    "multi layer perceptron 1" : MLPRegressor(hidden_layer_sizes=100, activation="identity"),
    "multi layer perceptron 2" : MLPRegressor(hidden_layer_sizes=100, activation="logistic"),
    "multi layer perceptron 3" : MLPRegressor(hidden_layer_sizes=100, activation="tanh"),
    "multi layer perceptron 4" : MLPRegressor(hidden_layer_sizes=100, activation="relu"),
    "multi layer perceptron 5" : MLPRegressor(hidden_layer_sizes=150, activation="identity"),
    "multi layer perceptron 6" : MLPRegressor(hidden_layer_sizes=150, activation="logistic"),
    "multi layer perceptron 7" : MLPRegressor(hidden_layer_sizes=150, activation="tanh"),
    "multi layer perceptron 8" : MLPRegressor(hidden_layer_sizes=150, activation="relu"),
    "Adaboost 1" : AdaBoostRegressor(n_estimators=50, loss="linear"),
    "Adaboost 2" : AdaBoostRegressor(n_estimators=50, loss="square"),
    "Adaboost 3" : AdaBoostRegressor(n_estimators=50, loss="exponential"),
    "Adaboost 4" : AdaBoostRegressor(n_estimators=100, loss="linear"),
    "Adaboost 5" : AdaBoostRegressor(n_estimators=100, loss="square"),
    "Adaboost 6" : AdaBoostRegressor(n_estimators=100, loss="exponential"),
    "Ridge 1" : Ridge(alpha=0.25),
    "Ridge 2" : Ridge(alpha=0.5),
    "Ridge 3" : Ridge(alpha=1),
    "Ridge 4" : Ridge(alpha=1.5),
    "Ridge 5" : Ridge(alpha=2),
}

In [41]:
# Function to get and pre-process the datas
def get_data(frac: float = 1.0) -> Tuple:
    """Function used for the weather dataset"""

    data = pd.read_csv(DATA_PATH).sample(frac=frac, random_state=RANDOM_STATE)
    target_column = "salary_in_usd"
    data = data.drop(["salary_currency", "salary"], axis=1)
    for column in ["experience_level", "employment_type", "job_title", "employee_residence", "company_location", "company_size"]:
        data[column] = LabelEncoder().fit_transform(data[column])
    data = data.dropna(axis=0)

    iforest = IsolationForest(contamination=0.1, random_state=RANDOM_STATE)
    outliers = iforest.fit_predict(data)
    clean_data = data[(outliers != -1)]

    # we normalize
    min_max_scaler = MinMaxScaler()
    clean_array = min_max_scaler.fit_transform(clean_data)
    clean_data = pd.DataFrame(clean_array, columns=clean_data.keys())

    data_values = clean_data.drop([target_column], axis=1)
    data_target = clean_data[target_column]
    x_train, x_test, y_train, y_test = train_test_split(
        data_values, data_target, test_size=0.3, random_state=RANDOM_STATE
    )
    return (x_train, y_train), (x_test, y_test)

In [42]:
def log_params(X_train: np.ndarray, X_test: np.ndarray, model_name: str) -> None:
    mlflow.log_param("nb_features", X_train.shape[1])
    mlflow.log_param("nb_samples_train", X_train.shape[0])
    mlflow.log_param("nb_samples_test", X_test.shape[0])
    mlflow.log_param("model_name", model_name)
    mlflow.log_param("model_class", type(MODEL_FUNCTION[model_name]))

In [43]:
def main() -> None:
    mlflow.set_tracking_uri(URI)
    mlflow.sklearn.autolog()
    frac = 1
    print("data loading")
    (X_train, Y_train), (X_test, Y_test) = get_data(frac)
    for model_name in MODEL_FUNCTION:
        run_name = f"Run of {model_name}"
        with mlflow.start_run(run_name=run_name, experiment_id=EXPERIMENT_ID):
            model = MODEL_FUNCTION[model_name]
            model.fit(X_train, Y_train)
            mlflow.sklearn.log_model(model, "model")
            log_params(X_train, X_test, model_name)  # this line is optional
            model_uri = mlflow.get_artifact_uri("model")
            eval_data = X_test
            eval_data["label"] = Y_test
            mlflow.evaluate(
                model=model_uri,
                data=eval_data,
                targets="label",
                model_type="regressor",
                evaluators="default",
            )
            mlflow.end_run()

In [44]:
main()  # We launch it all

data loading


2023/05/25 14:42:03 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '82ccbf6219f6459099bba8278bb18c78', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
X does not have valid feature names, but IsolationForest was fitted with feature names
2023/05/25 14:42:03 WARNING mlflow.sklearn: Training metrics will not be recorded because training labels were not specified. To automatically record training metrics, provide training labels as inputs to the model training function.
2023/05/25 14:42:03 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\hennecarta\Anaconda3\lib\site-packages\mlflow\models\signature.py:130: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schem